In [1]:
from elasticsearch import Elasticsearch
# es = Elasticsearch("http://localhost:9200",
# basic_auth = ('elasticsearch', 'e3TKzHmKRFWBP4gY--cjeQ'),
# ca_certs = 'C:/Users/lenovo/Downloads/elasticsearch-9.1.3-windows-x86_64/elasticsearch-9.1.3/config/certs/http_ca.crt'
# )
es = Elasticsearch("http://localhost:9200",
                   basic_auth = ('elasticsearch', 'e3TKzHmKRFWBP4gY--cjeQ')
                   )
es.ping() 

True

## Prepare the data

In [2]:
import pandas as pd
df = pd.read_csv('marathi_dataset.csv')
print(df.shape)
df.head()

(30, 7)


,scheme_id,scheme_name,description,department,category,eligibility,deadline
0,1,शेतकरी अनुदान योजना,बियाण्यांसाठी शेतकऱ्यांना आर्थिक मदत देणारी योजना,कृषी विभाग,अनुदान,जमीन धारक शेतकरी,3/31/2024
1,2,महिला स्वसहाय्य गट योजना,महिला स्वसहाय्य गटांना व्यवसाय सुरू करण्यासाठी...,महिला व बालकल्याण विभाग,कर्ज सबसिडी,महिला स्वसहाय्य गटाचा सदस्य,6/15/2024
2,3,विद्यार्थी शिष्यवृत्ती योजना,शैक्षणिक कामगिरीच्या आधारे विद्यार्थ्यांना शिष...,शिक्षण विभाग,शिष्यवृत्ती,"मागास वर्गीय विद्यार्थी, ७५% पेक्षा जास्त गुण",4/30/2024
3,4,राज्य शासकीय नोकरी अर्ज,राज्य शासकीय सेवेसाठी अर्ज करण्याची ऑनलाइन प्र...,सामान्य प्रशासन विभाग,नोकरी,"१८-३८ वयोगट, संबंधित पात्रता",5/20/2024
4,5,मुफ्त आरोग्य तपासणी शिबिर,ग्रामीण भागात रहिवाशांसाठी विनामूल्य आरोग्य तप...,आरोग्य विभाग,आरोग्य सेवा,सर्व नागरिक,7/10/2024


In [3]:
df.isna().sum()

scheme_id      0
scheme_name    0
description    0
department     0
category       0
eligibility    0
deadline       0
dtype: int64

### Converting description field to vector using sbert model

In [4]:
import numpy as np
print(np.__version__)


2.3.2


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('l3cube-pune/marathi-sentence-similarity-sbert')

In [6]:
df["description_vector"] = df["description"].apply(lambda x: model.encode(x))

In [7]:
df.head()

,scheme_id,scheme_name,description,department,category,eligibility,deadline,description_vector
0,1,शेतकरी अनुदान योजना,बियाण्यांसाठी शेतकऱ्यांना आर्थिक मदत देणारी योजना,कृषी विभाग,अनुदान,जमीन धारक शेतकरी,3/31/2024,"[-0.019805217, -0.018249715, -0.012199402, 0.0..."
1,2,महिला स्वसहाय्य गट योजना,महिला स्वसहाय्य गटांना व्यवसाय सुरू करण्यासाठी...,महिला व बालकल्याण विभाग,कर्ज सबसिडी,महिला स्वसहाय्य गटाचा सदस्य,6/15/2024,"[-0.026199086, 0.005875388, -0.0020529723, 0.0..."
2,3,विद्यार्थी शिष्यवृत्ती योजना,शैक्षणिक कामगिरीच्या आधारे विद्यार्थ्यांना शिष...,शिक्षण विभाग,शिष्यवृत्ती,"मागास वर्गीय विद्यार्थी, ७५% पेक्षा जास्त गुण",4/30/2024,"[-0.033258807, -0.0118139135, -0.0060276566, 0..."
3,4,राज्य शासकीय नोकरी अर्ज,राज्य शासकीय सेवेसाठी अर्ज करण्याची ऑनलाइन प्र...,सामान्य प्रशासन विभाग,नोकरी,"१८-३८ वयोगट, संबंधित पात्रता",5/20/2024,"[-0.016152378, -0.012807344, 0.019232813, 0.01..."
4,5,मुफ्त आरोग्य तपासणी शिबिर,ग्रामीण भागात रहिवाशांसाठी विनामूल्य आरोग्य तप...,आरोग्य विभाग,आरोग्य सेवा,सर्व नागरिक,7/10/2024,"[-0.013903189, -0.014410155, 0.0020151383, -0...."


In [8]:
es.ping()

True

### create new index in elasticsearch

In [10]:
from indexMappings import indexMappings
es.indices.create(index = "marathi_schemes" , mappings = indexMappings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [marathi_schemes/hrsRRFOBQOS5AipiyOTylQ] already exists')

### insert data into index

In [11]:
record_list = df.to_dict("records")

In [12]:
record_list[0]

{'scheme_id': 1,
 'scheme_name': 'शेतकरी अनुदान योजना',
 'description': 'बियाण्यांसाठी शेतकऱ्यांना आर्थिक मदत देणारी योजना',
 'department': 'कृषी विभाग',
 'category': 'अनुदान',
 'eligibility': 'जमीन धारक शेतकरी',
 'deadline': '3/31/2024',
 'description_vector': array([-1.98052172e-02, -1.82497147e-02, -1.21994019e-02,  6.73947576e-03,
         6.23329217e-03, -1.99643485e-02, -3.30120442e-03,  6.63491711e-03,
         2.25269478e-02,  9.08388942e-03,  1.19500216e-02,  1.84499547e-02,
        -2.02964135e-02, -9.90161393e-03, -1.09337056e-02, -8.66110437e-03,
        -1.70170031e-02, -5.73694613e-03, -2.06286479e-02,  1.10067381e-02,
        -1.62264111e-03,  7.21476367e-03,  3.61555209e-03, -5.10177761e-03,
        -1.27575099e-02,  1.07107917e-02, -6.86461478e-03, -2.08935398e-03,
         6.85015181e-03,  8.62135738e-03, -2.05656160e-02,  8.20860546e-03,
         5.15536917e-03,  2.72035953e-02, -1.12600727e-02, -4.32745001e-04,
        -4.51561762e-03,  5.05922071e-04,  1.12418588e-

### convert all records into index in elastic search

In [13]:
from datetime import datetime

for record in record_list:
    try:
        if 'deadline' in record:
            val = record['deadline']
            # Convert only if it's a valid date
            try:
                # Parse MM/DD/YYYY and convert to YYYY-MM-DD
                record['deadline'] = datetime.strptime(val, "%m/%d/%Y").strftime("%Y-%m-%d")
            except:
                # If it's not a date (like "चालू"), set null
                record['deadline'] = None

        es.index(index="marathi_schemes", document=record, id=record['scheme_id'])
    except Exception as e:
        print("error", e)


In [16]:
es.count(index="marathi_schemes")

ObjectApiResponse({'count': 30, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## search in data

In [17]:
input_text = "आरोग्य"
vector = model.encode(input_text)

query = {
    "size": 2,
    "knn": {
        "field": "description_vector",
        "query_vector": vector.tolist(),
        "k": 2,
        "num_candidates": 10
    }
}

result = es.search(index="marathi_schemes", body=query, source=["scheme_name", "description"])
result['hits']['hits']

C:\Users\lenovo\AppData\Local\Temp\ipykernel_23380\3894436778.py:14: DeprecationWarning: Received 'source' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  result = es.search(index="marathi_schemes", body=query, source=["scheme_name", "description"])


[{'_index': 'marathi_schemes',
  '_id': '5',
  '_score': 0.8631873,
  '_source': {'scheme_name': 'मुफ्त आरोग्य तपासणी शिबिर',
   'description': 'ग्रामीण भागात रहिवाशांसाठी विनामूल्य आरोग्य तपासणी शिबिर'}},
 {'_index': 'marathi_schemes',
  '_id': '22',
  '_score': 0.8544108,
  '_source': {'scheme_name': 'वृक्षारोपण अभियान',
   'description': 'जागतिक आरोग्यासाठी मोठ्या प्रमाणावर वृक्षारोपण करणे'}}]